# Datenvalidierung mit Voluptuous (Schemadefinitionen)

In diesem Notebook verwenden wir [Voluptuous](https://github.com/alecthomas/voluptuous), um Schemata für unsere Daten zu definieren. Wir können dann die Schemaprüfung an verschiedenen Stellen unserer Bereinigung verwenden, um sicherzustellen, dass wir die Kriterien erfüllen. Schließllich können wir Ausnahmen für die Schemaüberprüfung verwenden, um unreine oder ungültige Daten zu markieren, beiseite zu legen oder zu entfernen.

## 1. Importe

In [1]:
import logging
import pandas as pd
from datetime import datetime
from voluptuous import Schema, Required, Range, All, ALLOW_EXTRA
from voluptuous.error import MultipleInvalid, Invalid

## 2. Logger

In [3]:
logger = logging.getLogger(0)
logger.setLevel(logging.WARNING)

## 3. Beispieldaten lesen

In [4]:
sales = pd.read_csv('https://raw.githubusercontent.com/kjam/data-cleaning-101/master/data/sales_data.csv')

## 4. Daten untersuchen

In [5]:
sales.head()

,Unnamed: 0,timestamp,city,store_id,sale_number,sale_amount,associate
0,0,2018-09-10 05:00:45,Williamburgh,6,1530,1167.0,Gary Lee
1,1,2018-09-12 10:01:27,Ibarraberg,1,2744,258.0,Daniel Davis
2,2,2018-09-13 12:01:48,Sarachester,2,1908,266.0,Michael Roth
3,3,2018-09-14 20:02:19,Caldwellbury,14,771,-108.0,Michaela Stewart
4,4,2018-09-16 01:03:21,Erikaland,11,1571,-372.0,Mark Taylor


In [6]:
sales.dtypes

Unnamed: 0       int64
timestamp       object
city            object
store_id         int64
sale_number      int64
sale_amount    float64
associate       object
dtype: object

## 5. Schema definieren

In [7]:
schema = Schema({
    Required('sale_amount'): All(float, 
                                 Range(min=2.50, max=1450.99)),
}, extra=ALLOW_EXTRA)

In [8]:
error_count = 0
for s_id, sale in sales.T.to_dict().items():
    try:
        schema(sale)
    except MultipleInvalid as e:
        logging.warning('issue with sale: %s (%s) - %s', 
                        s_id, sale['sale_amount'], e)
        error_count += 1

In [9]:
error_count

69

In [10]:
sales.shape

(213, 7)

Aktuwll wissen wir jedoch noch nicht, ob

* wir ein falsch definiertes Schema haben
* möglicherweise negative Werte zurückgegeben oder falsch markiert werden
* höhere Werte kombinierte Einkäufe oder Sonderverkäufe sind

## 6. Hinzufügen einer benutzerdefinierten Validierung

In [11]:
def ValidDate(fmt='%Y-%m-%d %H:%M:%S'):
    return lambda v: datetime.strptime(v, fmt)

In [12]:
schema = Schema({
    Required('timestamp'): All(ValidDate()),
}, extra=ALLOW_EXTRA)

In [13]:
error_count = 0
for s_id, sale in sales.T.to_dict().items():
    try:
        schema(sale)
    except MultipleInvalid as e:
        logging.warning('issue with sale: %s (%s) - %s', 
                        s_id, sale['timestamp'], e)
        error_count += 1

In [14]:
error_count

0

## 7. Gültige Datumsstrukturen sind noch keine gültigen Daten

In [15]:
def ValidDate(fmt='%Y-%m-%d %H:%M:%S'):
    def validation_func(v):
        try:
            assert datetime.strptime(v, fmt) <= datetime.now()
        except AssertionError:
            raise Invalid('date is in the future! %s' % v)
    return validation_func

In [16]:
schema = Schema({
    Required('timestamp'): All(ValidDate()),
}, extra=ALLOW_EXTRA)

In [17]:
error_count = 0
for s_id, sale in sales.T.to_dict().items():
    try:
        schema(sale)
    except MultipleInvalid as e:
        logging.warning('issue with sale: %s (%s) - %s', 
                        s_id, sale['timestamp'], e)
        error_count += 1

In [18]:
error_count

0